### Simple BTC TRADING BOT ###

In [53]:
from binance import Client  # Import the Client class from the binance module
import csv  # Import the csv module
import pandas as pd  # Import the pandas library 
from datetime import datetime as dt  # Import the datetime class from the datetime module and alias it as dt (it depends on your choice )
client=Client(None,None)  # Create a Binance client instance with None as API key and secret key


In [46]:
# create a new function to get data for the specified symbol, period, start date, and end date
def get_data(symbol, period, start_date, end_date):
    # Get historical klines data for the specified symbol, period, start date, and end date
    data_coin = client.get_historical_klines(symbol, period, start_date, end_date)
    # Return the obtained data
    return data_coin


In [47]:
def create_csv(symbol, data_coin): # create new function our csv file
    # open a csv file for writing with the specified symbol in the filename
    csvfile = open(symbol + 'CSV', 'w', newline='')
    # create a csv  writer object with comma as the delimiter
    writer = csv.writer(csvfile, delimiter=',')
    # iterate over each row of data in the data_coin list
    for data1 in data_coin:
        # write each row of data to the csv file
        writer.writerow(data1)
    # close the csv file
    csvfile.close()

In [48]:
create_csv('BTCUSDT',get_data('BTCUSDT',client.KLINE_INTERVAL_1DAY,'10 March 2018','10 March 2024')) # call the above function

In [54]:
# define a function to create a dataFrame from a csv file
def create_dataframe(filename, columns):
    # read the csv file into a DataFrame with the specified column names
    df = pd.read_csv(filename, names=columns)
    # Return the DataFrame
    return df

# Define the column names for the dataFrame
cols = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Qavol', 'Numoftr', 'Tabuy_volume', 'Takquoteassetvolume', 'Ignore']

# make sure 'BTCUSDT.csv' is the correct filename and it exists in your directory
# Create a DataFrame using the create_dataframe function
df = create_dataframe('BTCUSDTCSV', cols)
# print the DataFrame
print(df)


          Open_time      Open      High       Low     Close        Volume  \
0     1520640000000   9230.00   9490.00   8667.07   8770.22  37180.012857   
1     1520726400000   8770.22   9740.00   8450.00   9533.57  44325.973386   
2     1520812800000   9533.57   9888.88   8780.00   9131.34  42230.777930   
3     1520899200000   9131.34   9474.00   8823.00   9150.00  40191.409358   
4     1520985600000   9151.92   9333.78   7900.28   8170.00  49708.094108   
...             ...       ...       ...       ...       ...           ...   
2188  1709683200000  63724.01  67641.10  62779.14  66074.04  78738.854910   
2189  1709769600000  66074.04  67980.00  65551.00  66823.17  53059.886900   
2190  1709856000000  66823.18  69990.00  66082.66  68124.19  74261.932842   
2191  1709942400000  68124.20  68541.10  67861.10  68313.27  19872.897430   
2192  1710028800000  68313.28  69887.61  68094.75  68955.88  38404.668350   

         Close time         Qavol  Numoftr  Tabuy_volume  Takquoteassetvolu

In [51]:
from datetime import datetime as dt
# converting datetime
def convert_time(timestamp):
    return dt.fromtimestamp(timestamp / 1000)  # Since Binance trade time works in milliseconds, we need to divide this by 1000 and in this case we will reduce the timestamp by 3 digits.


In [56]:
import pandas_ta as ta  # import the pandas_ta library with the alias ta

def Golden_BACKtest():  # Define a function named Golden_BACKtest
    wallet = 100  # Initialize the wallet balance to 100
    count_of_buy = 0  # Initialize the count of buy orders to 0
    count_of_sells = 0  # Initialize the count of sell orders to 0
    sum_of_coin = 0  # Initialize the sum of coins to 0
    comission_percentage = 75 / 10000  # Calculate the commission percentage
    my_csv = 'BTCUSDTCSV'  # Set the CSV file name
    headings = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume',  # Define column headings for the csv file
                'Close time', 'Qavol', 'Numoftr', 'Tabuy_volume', 'Takquoteassetvolume', 'Ignore']
    df = pd.read_csv(my_csv, names=headings)  # Read the CSV file into a DataFrame
    # extract the  columns from the DataFrame
    open = df['Open']  
    close = df['Close']  
    high = df['High']  
    low = df['Low'] 
    open_time = df['Open_time']  
    close_time = df['Close time'] 
    sma50 = ta.ma('sma', close, length=50)  # Calculate the 50-day simple moving average
    sma200 = ta.ma('sma', close, length=200)  # Calculate the 200-day simple moving average

    print(' ******  STARTING *******')  # Print a message indicating the start of the process
    for i in range(len(close)):  # Iterate over each data point in the 'Close' column
        # Check if sma50 and sma200 are not None
        if pd.isna(sma50[i]) is False and pd.isna(sma200[i]) is False:
            # Check if the condition for buying is met
            if sma50[i - 1] < sma200[i - 1] and sma50[i] > sma200[i]: ### --->>>  positive crossover <<<<-
                # Print a message indicating the buy order
                print(convert_time(open_time[i + 1]), 'is that time', wallet / close[i], ' BUYING BTC')
                print('********')  # Print a separator
                count_of_buy += 1  # Increment the count of buy orders
                sum_of_coin = wallet / close[i]  # Calculate the sum of coins
                payyed_comission = comission_percentage * wallet  # Calculate the commission paid
            # check if the condition for selling is met
            if sma50[i - 1] > sma200[i - 1] and sma50[i] < sma200[i] and count_of_buy > 0: ### --->>>  "negative crossover <<<<-
                # Print a message indicating the sell order
                print(convert_time(open_time[i + 1]), 'is that time', sum_of_coin, ' selling BTC')
                print('********')  # print a separator
                count_of_sells += 1  # increment the count of sell orders
                price = close[i] * sum_of_coin  # Calculate the total price
                wallet = price  # Update the wallet balance
                sum_of_coin = 0  # Reset the sum of coins
                payyed_comission = comission_percentage + wallet  # Calculate the commission paid
                print(f'Last result: {wallet}')  # Print the last result
    print(f'Sum of proceddure:{count_of_buy + count_of_sells}')  # Print the sum of procedures
    print(f'Sum of comission {payyed_comission}')  # Print the sum of commission
    print(f'Sum of wallet {wallet}')  # Print the sum of the wallet

# Call the function to execute
Golden_BACKtest()


    

 ******  STARTING *******
2019-04-25 04:00:00 is that time 0.018467220683287166  BUYING BTC
********
2019-10-27 04:00:00 is that time 0.018467220683287166  selling BTC
********
Last result: 170.45244690674053
2020-02-19 04:00:00 is that time 0.016769041803134623  BUYING BTC
********
2020-03-26 04:00:00 is that time 0.016769041803134623  selling BTC
********
Last result: 111.97410280750523
2020-05-22 04:00:00 is that time 0.012347383878251475  BUYING BTC
********
2021-06-20 04:00:00 is that time 0.012347383878251475  selling BTC
********
Last result: 438.1311122683894
2021-09-16 04:00:00 is that time 0.009104702299213372  BUYING BTC
********
2022-01-15 04:00:00 is that time 0.009104702299213372  selling BTC
********
Last result: 392.0481168160358
2023-02-08 04:00:00 is that time 0.01686920641054591  BUYING BTC
********
2023-09-13 04:00:00 is that time 0.01686920641054591  selling BTC
********
Last result: 435.9019805691474
2023-10-31 04:00:00 is that time 0.012644101362625533  BUYING BT

Conclusion




The project involved backtesting a trading strategy known as the "Golden Cross" on historical Bitcoin price data. The strategy identifies buy and sell signals based on the crossover of two moving averages: the 50-day simple moving average (SMA50) and the 200-day simple moving average (SMA200).

The backtesting process was initiated, and a series of buy and sell signals were generated based on the Golden Cross criteria. Each signal included the timestamp of the event, the action taken (buying or selling BTC), and the resulting wallet balance in USD after the transaction.

The analysis revealed several instances where the Golden Cross signal prompted buying BTC, followed by subsequent sell signals. Each transaction resulted in changes to the wallet balance, reflecting the profitability of the trading strategy.

Overall, the backtesting process executed a total of 11 trading procedures during the specified period. The total commission paid for these transactions amounted to 3.2692648542686054 USD. The project concluded with a final wallet balance of 435.9019805691474 USD.
total profit=435.9019805691474 -100 =335.90128...